In [1]:
# import os for creating directories and file paths in the notebook environment
import os

In [2]:
# checking the current working directory of the notebook environment
%pwd

'g:\\Jagadish\\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS\\research'

In [3]:
# changing the current working directory to the parent directory of the notebook environment
os.chdir("../")

In [4]:
# checking the current working directory of the notebook environment
%pwd

'g:\\Jagadish\\Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS.mlflow"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    """
    Data class to hold configuration settings for model evaluation.

    Attributes:
        path_of_model (Path): Path to the trained model.
        training_data (Path): Path to the training data used for the model.
        all_params (dict): Dictionary containing all parameters used for model training and evaluation.
        mlflow_uri (str): URI for MLflow tracking server to log experiments and results.
        params_image_size (list): List containing the dimensions of the input images.
        params_batch_size (int): Size of the batches for evaluation.
    """
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

[2024-12-16 19:21:02,392: INFO: __init__: Logging setup is complete.]


In [10]:
class ConfigurationManager:
    """
    Class to manage the configuration and parameters for setting up the evaluation environment.

    Attributes:
        config_filepath (str): Path to the configuration YAML file.
        params_filepath (str): Path to the parameters YAML file.
    """

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """
        Initializes the ConfigurationManager with the given file paths.

        Args:
            config_filepath (str): Path to the configuration YAML file.
            params_filepath (str): Path to the parameters YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        create_directories([self.config.artifacts_root])  # Create directories specified in the config

    def get_evaluation_config(self) -> EvaluationConfig:
        """
        Retrieves the evaluation configuration.

        Returns:
            EvaluationConfig: An instance of EvaluationConfig with the specified settings.
        """
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",  # Path to the trained model
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",  # Path to the training data
            mlflow_uri="https://dagshub.com/jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS.mlflow",  # MLflow tracking URI
            all_params=self.params,  # Parameters from the parameters YAML file
            params_image_size=self.params.IMAGE_SIZE,  # Image size parameter
            params_batch_size=self.params.BATCH_SIZE  # Batch size parameter
        )
        return eval_config


In [11]:
import tensorflow as tf
from pathlib import Path
import dagshub
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    """
    Class to handle the evaluation process of a trained machine learning model.

    Attributes:
        config (EvaluationConfig): Configuration settings for model evaluation.
    """

    def __init__(self, config: EvaluationConfig):
        """
        Initializes the Evaluation with the given configuration.

        Args:
            config (EvaluationConfig): Configuration settings for model evaluation.
        """
        self.config = config

    def _valid_generator(self):
        """
        Creates a validation data generator using Keras' ImageDataGenerator.

        This method sets up the data augmentation and preprocessing steps and
        creates a flow from the directory specified in the configuration.
        """
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """
        Loads a trained Keras model from the specified file path.

        Args:
            path (Path): Path to the trained model file.

        Returns:
            tf.keras.Model: Loaded Keras model.
        """
        return tf.keras.models.load_model(path)

    def evaluation(self):
        """
        Evaluates the trained model using the validation data generator.

        This method loads the model, sets up the validation data generator,
        performs evaluation, and saves the evaluation scores.
        """
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        """
        Saves the evaluation scores to a JSON file.

        This method creates a dictionary of scores and saves it as a JSON file.
        """
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        """
        Logs the evaluation parameters and metrics into MLflow.

        This method logs the parameters, evaluation scores, and registers the
        model in MLflow if applicable.
        """
        # Initialize DagsHub for MLflow tracking
        dagshub.init(repo_owner='jagadishmali567', 
                     repo_name='Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS', 
                     mlflow=True)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Register the model
            mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")


In [13]:
try:
    # Create an instance of ConfigurationManager
    config = ConfigurationManager()
    
    # Retrieve the evaluation configuration
    eval_config = config.get_evaluation_config()
    
    # Create an instance of Evaluation with the retrieved configuration
    evaluation = Evaluation(eval_config)
    
    # Perform the evaluation
    evaluation.evaluation()
    
    # Log the evaluation results into MLflow
    evaluation.log_into_mlflow()

except Exception as e:
    # Raise any exception that occurs during the process
    raise e


[2024-12-16 19:21:05,600: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-16 19:21:05,633: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-16 19:21:05,650: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 19s 2s/step - loss: 23.7682 - accuracy: 0.5686
[2024-12-16 19:21:25,347: INFO: common: json file saved at: scores.json]
[2024-12-16 19:21:26,455: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as jagadishmali567

[2024-12-16 19:21:26,463: INFO: helpers: Accessing as jagadishmali567]
[2024-12-16 19:21:27,656: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS "HTTP/1.1 200 OK"]
[2024-12-16 19:21:28,766: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo 
"jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS"

[2024-12-16 19:21:28,766: INFO: helpers: Initialized MLflow to track repo "jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS"]


Repository 
jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS 
initialized!

[2024-12-16 19:21:28,780: INFO: helpers: Repository jagadishmali567/Production-Ready-Chest-Cancer-Detection-Deep-Learning-Model-MLOps-with-MLflow-DVC-CI-CD-and-AWS initialized!]


2024/12/16 19:21:30 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-16 19:21:33,214: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\arjun\AppData\Local\Temp\tmpsjl_f0gs\model\data\model\assets
[2024-12-16 19:21:35,517: INFO: builder_impl: Assets written to: C:\Users\arjun\AppData\Local\Temp\tmpsjl_f0gs\model\data\model\assets]


c:\Users\arjun\anaconda3\envs\ccancer\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2024/12/16 19:22:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
